<span type="title">模板方法模式和迭代器、组合模式</span> | <span type="update">2018-09-08</span> - Version <span type="version">1.3</span>
    
    
<span type="intro"><p class="card-text">本章首先介绍了模板方法模式。模板方法通过在抽像层定义一组算法和服务流程，然后把这些步骤的实现推迟到子类进行，由子类选择实现或者重写抽象类的方法以提供自定义的服务。通过使用不同的关键字： final，protected，private，abstract，有效的指明了抽象类和子类的职责。此外，模板方法的算法可以有不同的分支，子类通过钩子进行控制。模板方法和工厂方法类似，都是交给子类，而策略模式则是使用组合，不过这三者都属于提供服务的目的（区别于上一张的适配器、装饰器、命令模式实现接口标准化的目的）。</p><p class="card-text">在第二部分，着重介绍了迭代器模式和组合模式，迭代器模式的中心思想是：为每个类分配单一的责任以简化接口调用。迭代器模式强调通过分离责任简化对于获取容器元素的接口的调用。</p><p class="card-text">组合模式一般用来保存一组相同的类实例对象，不同于策略模式，组合模式保存对象后自己实现了其保存对象的类的接口，也就是说，组合模式通过实现其保存类的接口，导致其内部容器实例可以保存类和类的组合（主类对象本身）这样一种奇妙的现象。组合模式违反了OOP每个类只有一种变化原因的原则，牺牲了透明性，统一了接口。调用主类的方法通常使用容器迭代器逐个返回其保存子类的方法，如果碰到子类是组合的情况，则递归调用迭代器返回其内部方法。迭代器和组合模式均属于提供标准接口这一大类的设计模式（组合+接口）。和其比较相似的有：装饰器模式、适配器模式、外观模式、命令模式、代理模式。</p></span>

# 模板方法模式

## 模板方法定义

模板方法模式指的是，在抽象类中搭建框架，定义算法的步骤，然后把这些步骤的实现推迟到子类的设计模式。

模板方法的类图如下：

![](c5p1.png)

**模板方法的关键词和责任划分**

模板方法模式很像工厂方法模式，其区别在于，前者交给子类的是“方法”（算法的某个步骤），后者交给子类的是“对象”（然后对对象调用各种方法完成服务）。模板方法模式可以使用 abstract 限制以完全交给子类，也可以使用 protected 限制以实现钩子，由子类决定是否要实现，或者使用默认值，还可以使用 private 关键字限制以禁止子类实现，统一由父类完成。

**模板方法的钩子和算法结构逻辑**

此处的钩子不是指由子类决定实现或者不实现的方法（如果不实现，直接return null;），然后这些方法放在算法的序列语句中，如果为空则跳过。此处的钩子指的是，在算法中的分支情况。比如：

```java
process() {
    methodA();
    methodB();
    if (methodC()) {
        methodAdd();
    }
    methodD()
}
```

这样可以极大的扩展算法的结构，产生很多的分支，然后由抽象类维护这个具体提供服务的流程，而由子类决定其各个步骤的具体实现，以及选择什么步骤进行实现。

## 示例代码

```java
import java.io.BufferedReader;
import java.io.InputStreamReader;

public class DrinkShop {
    public static void main(String[] args) {
        DrinkFactory coffee = new Coffee();
        coffee.prepareDrink();
        DrinkFactory tea = new Tea();
        tea.prepareDrink();
    }
}

abstract class DrinkFactory {
    public final void prepareDrink() {
        fatchWater();
        boiledWater();
        addStuff();
        addRelish();
        pourToPot();
        if (needPackUp()) {    packUp(); }
    }
    private void fatchWater() { System.out.println("Fetching some water now... ");}
    protected void boiledWater() { System.out.println("Boiled Water now..."); }
    protected void pourToPot() { System.out.println("Pour the drink into pot...");}
    protected abstract void addRelish();
    protected abstract void addStuff();
    protected Boolean needPackUp() { return false; }
    protected void packUp() { System.out.println("Packing now...");}
}
class Coffee extends DrinkFactory {
    @Override
    protected void boiledWater() { System.out.println("Boiled water to 100 degree..."); }
    @Override
    protected void addRelish() {System.out.println("Adding milk...");}
    @Override
    protected void addStuff() {System.out.println("Adding coffee...");}
    @Override
    protected Boolean needPackUp() { return true; }
}
class Tea extends DrinkFactory {
    @Override protected void addRelish() {
        BufferedReader reader = new BufferedReader(new InputStreamReader(System.in));
        System.out.println("Do you want to add some juice? y/n");
        try {
            String ans = reader.readLine();
            if (ans.toLowerCase().equals("y")) {
                System.out.println("Adding juice now...");
            } else throw new RuntimeException("I hate you");
        } catch (Exception e) {
            System.out.println("Ok, I will skip this step");
        } finally {
            System.out.println("Adding process done!");
        }
    }
    @Override protected void addStuff() {System.out.println("Adding tea leaf...");}
}
```

以上代码展示了咖啡和茶的制作过程，在其中遵循了“好莱坞原则”：在较低层次的类中不应该调用较高（较为抽象）层次的方法，而是应该在较为抽象的方法中调用较低层次的方法（通过继承隐式调用，通过RTTI、多态进行调用）。注意，此处较高层次调用较低层次不是说要在较高层次内部保留较低层次的类型，而是说，在较高层次调用抽像接口的时候，实际通过转型或者继承动态的调用了较低层次类的实现。至于较低层次不应该调用较高层次，这是指的是，你不应该在一个具体的类中保存任何抽像的类并调用它们任何的代码。而是应该提供一个静态的实现了抽像接口的方法，以供较高层次在它处通过构造具体类的方式进行调用。

注意到，在上述代码中使用了 protected 以允许子类提供自己的实现，使用了 private 方法以保证饮料的水源健康可靠。注意到我们是如何使用钩子让 addRelish 方法是在算法中进入别的分支，提供其他服务的。这里的“外带”不应该放入主分支，然后提供null，这样不合情理，外带是一个可选选项，而不是一个默认为空的必选项。注意，对于算法逻辑，使用 final 关键字保证子类可见但不可修改。

## OO原则（好莱坞原则）：别打给我，我会找你

定义：**高层次的类通过抽像接口构造具体示例以调用低层次的类以提供服务，但是低层次的类不能调用高层次的类。换句话说，由超类主控一切，当它们需要的时候，自然会去调用子类。**在子类中，不要调用超类，因为一旦超类和子类互相调用形成闭环，那么代码将难以修改（势必会互相影响）。

注意，策略模式并没有违反好莱坞原则。对于策略模式而言，其内部封装了一个抽像的实例，然后调用这个实例完成一系列操作。几个互相替代的策略类才是底层，其仅提供方法，而传入高层次的类内部的实例变量中，进行动态调用。

# 迭代器模式

## 迭代器模式的定义

迭代器模式指的是：**通过提供一种方法顺序访问一个聚合对象中的各个元素，而又不暴露其内部的表示。**迭代器模式通过将维护指针的任务交给迭代器，而避免了任何调用容器元素的对象的复杂实现。一般而言，迭代器只能正向游走，顺序取出元素，但是ListIterator可以正向和反向游走。

迭代器的实现需要以下元素：在一个类中实现了产生迭代器的接口方法，调用这个方法以产生迭代器对象。这个迭代器对象实现了迭代器接口，可以调用这个这个接口的方法以获取迭代器对象的元素。因此，大概是这样的层次：存储迭代器的对象 -> 迭代器对象 -> 迭代器的元素。其一需要实现获取迭代器接口方法，其二需要实现迭代器接口，通过接口湖片区其三：迭代器元素。其类图如下：

Client 保存 Aggregate 的实现 ConcreteAggregate，通过这个接口获取 Iterator 的实现：ConcreteIterator。

![](c5p2.png)

## OO设计原则：单一责任

**一个类应该只有一个引起变化的原因。**

我们要尽可能的避免类的代码的改变。（这听起来挺让人沮丧的，因为改变是创造的必要条件）。因此，为了避免改变的同时满足丰富多样的服务需求，我们需要为每个类划分单一的责任。如果一个类有多个责任，那么其在未来就更有可能被改变。而单一责任的类则没有这种烦恼，如果不能满足需求，直接被舍弃，而不用浪费任何的代码，不用将尚可使用的代码和不可使用的代码小心的进行区分。

**内部类的迭代器 OR 独立的迭代器 OR 主类充当迭代器**

迭代器很好的实现了单一责任，它仅用于维护一个指针，并且从容器中返回合适的对象，因此将其构造成一个类。一般而言，有时候我们会直接让主类实现Iterator的接口，完成迭代器类的责任，这样好不好呢？要看主类的责任是什么，如果主类没有什么责任，那么其代理迭代器可以理解，但是，如果其有别的责任，那么应该使用内部类保存和维护一个迭代器，使用一个方法来获取这个迭代器对象。如果这个迭代器很大，或者具有其他类也可以复用的代码，那么将这个迭代器从主类中独立出去进行复用。但是一般而言，迭代器由于和主类过于密切，放在一个类中更容易管理和维护类。（另外使用内部类的原因是，迭代器放在内部可以容易的访问外部容器进行迭代，而放在外部的话，则需将容器先传递到迭代器内部，然后再进行遍历。）



# 组合模式

组合模式的定义是：**将对象组合成树形结构来表现“整体/部分”层次结构。组合能让客户以一致的方式处理个别对象以及对象组合。**换句话说，一个菜单是整体，菜单项是部分，但是这个菜单包含了一个子菜单，对于这个子菜单的菜单项，它是整体。组合模式对于一个菜单中的项目一视同仁，不论它是菜单项还是子菜单，这样虽然可能导致混淆，但是却更加的一致。

组合模式的类图如下：

![](c5p3.png)

组合模式将Leaf和Composite看作相同的元素看待，统统实现Component接口，这样的话，对于外界而言，这个组合就是透明的。这样虽然违反了“一个类只应该有一个责任”这一原则，但是换取了一致性和透明性，因此还是值得的。

组合模式和迭代器模式的关系：组合模式中可以实现迭代器，如果一个组合包含一个组合，那么使用递归，让一个迭代器从另一个迭代器中返回元素，到最后完全进行遍历。

总而言之，组合模式抹除了组合中各元素的差异性，使用统一的接口来处理客户端调用。使用类责任换取了高的透明度。

## 组合模式和工厂模式

组合模式一般组合有几个对象，并且同时实现了对象类的接口，因此，可以把自己伪装成为对象，这样的话，组合模式内部容器可以保存对象、对象组合，其中对象组合又可以递归保存对象和对象组合。组合模式使用透明性换取统一性。**组合模式常常用于组合几个相同的类别的对象，批量处理提供服务。**

组合模式和工厂模式类似，常常用于“组合”几个实例。但是，组合模式更倾向于对其内部的实例对象逐个遍历，提供方法服务，而抽像工厂则更侧重于暴露和提供对象本身。抽像工厂并不提供服务，而工厂方法则和组合模式的目的一致，但是，工厂方法模式是通过交给子类以及继承，而不是组合+委托的方式实现提供服务的目的的。

# 提供服务 vs 接口标准化

和上一章不同，这一章的这些方法更偏向“提供服务”，而不是“实现接口的标准化”。为了避免提供服务时的耦合，我们需要遵循在超类中定义服务算法，在子类中提供算法步骤的具体实现的方式，由超类调用子类完成不同的服务。

提供服务分为两大类型：交给子类，算法拆分，使用继承复用代码。或者使用组合，将算法合并成策略，设置为状态，运行时可以更改，更有弹性。模板方法和工厂方法都是选择了使用继承，交给子类，而策略模式则更强调策略、整合和组合。

## 交给子类 OR 使用组合：策略，状态、模板和工厂

模板方法模式和工厂方法模式是典型的“交给子类”。通过继承，工厂方法的子类定义了对象，使用由抽象超类提供的服务算法完成服务。而模板方法的子类并没有提供对象，而是提供了算法的一个部分，然后使用由抽象超类提供的服务算法完成服务。

策略模式也是为了“提供服务”，不过和“交给子类不同”，其采用的是组合的方式，将互相替代的算法通过外部传递并且保存成具体实例，然后调用这个实例完成服务。策略模式的算法更适合互相替代的算法，而不是庞大的，需要切分并且每个步骤不同的算法（比如选择一种加密方式，这些加密方式互相替代，使用策略模式更合适。再比如客户端连接服务器，需要经过一系列的层：物理层、链路层、网际层、应用层是顺序结构，但是各个层可选的标准不同，这就更适合模板方法）。

当然，策略模式和模板方法模式实际更为相似。我们可以为定义几组不同的连接方式，比如策略A，物理层使用以太网，链路层使用IP，网际层使用TCP，应用层选择HTTP。策略B，物理层使用WLAN，链路层使用IP，网际层使用TCP，应用层使用HTTP。可以将算法拆分，交给子类使用继承，也可以将算法合并成几种情况，形成策略，然后使用组合。

策略模式可以在运行时更改策略，灵活性更好。依赖程度更低。但是模板方法模式重用了很多代码，效率更高，对于庞大的算法，分层的结构更清晰。而策略模式面对庞大的算法可能有大量的不同策略，略显僵硬。

## 统一接口 OR 简化接口：命令、适配、外观、装饰、迭代器和组合

这一大类属于接口标准化的设计模式。其中，命令、适配、外观、装饰、迭代器和组合较为常见。

- 命令模式：多个对象对一个单独的接口方法，强调简化接口，严密封装
- 外观模式：多个对象对和本身不同的一组接口，强调简化接口，但不封装
- 适配模式：一个对象对和本身不同的一组接口，强调建立标准联系
- 装饰模式：多个对象对其本身相同的一组接口，强调添加责任
- 迭代模式：不确定个对象对一组特定接口，通过单一责任原则分离类的功能以简化接口调用，用于容器元素获取，强调简化接口，严密封装
- 组合模式：多个有交集接口的抽像，抽像成一个标准的接口，强调简化接口，突出透明性，但是牺牲了单一责任原则